In [1]:
import sys
sys.path.append('..')

import utils, selex_dca, indep_sites
import adabmDCA
import selex_distribution, energy_models, tree, data_loading, training

import torch
from utils import one_hot
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
from adabmDCA.utils import get_device, get_dtype

/Users/scrotti/Aptamer2025py/selex_dca.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
device = get_device("")
dtype = get_dtype("float64")

Running on CPU


In [6]:
L = 45
q = 4
n_rounds = 3
M = 100

sequences = [torch.randint(q, (M,L)) for _ in range(n_rounds)]
sequences_oh = [one_hot(seq).to(dtype=dtype, device=device) for seq in sequences]
total_reads = torch.Tensor([s.shape[0] for s in sequences_oh])

In [17]:
k = torch.randn(L, q, dtype=dtype, device=device)
h = torch.randn(L, q, dtype=dtype, device=device)
J = torch.randn(L, q, L, q, dtype=dtype, device=device)
mask = torch.ones(L, q, L, q)
mask[torch.arange(L), :, torch.arange(L), :] = 0
J = J * mask

tr = tree.Tree()
tr.add_node(-1)
tr.add_node(0)

selected_modes = torch.BoolTensor([[1],[1]])

Ns0 = energy_models.IndepSites(k)
potts = energy_models.Potts(J, h)
ps = selex_distribution.MultiModeDistribution(potts, normalized=False)
model = selex_distribution.MultiRoundDistribution(Ns0, ps, tr, selected_modes)

params = {"bias_Ns0": k, "bias_ps": h, "couplings_ps": J}

In [22]:
t = 2
x = sequences_oh[t]
params_t = selex_dca.get_params_at_round(params, t)

In [23]:
en_old = adabmDCA.statmech.compute_energy(x, params_t)
en_new = model.compute_energy_up_to_round(x, t)
(en_old - en_new).abs().max()

tensor(1.4211e-14, dtype=torch.float64, grad_fn=<MaxBackward1>)

In [24]:
%timeit adabmDCA.statmech.compute_energy(x, params_t)

106 μs ± 765 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [25]:
%timeit model.compute_energy_up_to_round(x, t)

189 μs ± 1.31 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [38]:
%load_ext line_profiler

In [43]:
selected_modes = torch.BoolTensor([[1, 0, 1],[1, 0, 0], [0, 1, 1]])

Ns0 = energy_models.IndepSites(k)
pottss = [energy_models.Potts(
    h = torch.randn(L, q, dtype=dtype, device=device),
    J = torch.randn(L, q, L, q, dtype=dtype, device=device))
    for _ in range(selected_modes.shape[1])]
ps = selex_distribution.MultiModeDistribution(*pottss, normalized=True)
model = selex_distribution.MultiRoundDistribution(Ns0, ps, tr, selected_modes)
%lprun -f selex_distribution.MultiModeDistribution.compute_energy model.compute_energy_up_to_round(x, t)

Timer unit: 1e-09 s

Total time: 0.000793 s
File: /Users/scrotti/Aptamer2025py/selex_distribution.py
Function: MultiModeDistribution.compute_energy at line 25

Line #      Hits         Time  Per Hit   % Time  Line Contents
    25                                               def compute_energy(
    26                                                       self,
    27                                                       x: torch.Tensor, # batch_size * L * q
    28                                                       selected: torch.BoolTensor, # n_rounds * n_modes
    29                                               ):
    30         1     713000.0 713000.0     89.9          minus_en_ = tuple(mode.compute_energy(x) for mode in self.modes)
    31         1      10000.0  10000.0      1.3          minus_en = torch.stack(minus_en_, dim=1)
    32         1       1000.0   1000.0      0.1          if self.normalized == True:
    33         1      27000.0  27000.0      3.4              minus_

In [27]:
M = 10**7

sequences = [torch.randint(q, (M,L)) for _ in range(n_rounds)]
sequences_oh = [one_hot(seq).to(dtype=dtype, device=device) for seq in sequences]
total_reads = torch.Tensor([s.shape[0] for s in sequences_oh])

In [28]:
batch_size = 10**6
datasets = [data_loading.SelexRoundDataset(seq_oh) for seq_oh in sequences_oh]
n_rounds = len(datasets) 
data_loaders = [torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True) for ds in datasets]

In [29]:
%timeit next(iter(data_loaders[t]))

1.68 s ± 385 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
batch_size = 10**6
datasets = [data_loading.SelexRoundDataset(seq_oh) for seq_oh in sequences_oh]
n_rounds = len(datasets) 
data_loaders = [torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=True) for ds in datasets]

In [46]:
%timeit next(iter(data_loaders[t]))

1.25 s ± 79.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
nseq = [sequences_oh[t].shape[0] for t in range(n_rounds)]
def load_batch(sequences_oh, t, nseq, batch_size):
    idx = torch.randperm(nseq[t])[:batch_size]
    sequences_oh[t][idx]

%timeit load_batch(sequences_oh, t, nseq, batch_size)

141 ms ± 542 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
